<a href="https://colab.research.google.com/github/PatrickMarinich/Mario-Kart-Stats-Project/blob/main/Kartnite_Stats_v3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Patch Notes

**Kartnite Stats** v3.1

*Written by: Patrick Marinich*

**Goal:** This scripts goal is to take user inputted *Mario Kart Wii* races and other stats and save them to a database to be used for calculations and analysis when user requested. 

**Current Functionalities**

*   Take in a user inputted race, with the track, players, and scores, and adds them to the database of all scores
*   Keep track of how many times each player playes on a singular track
* Get the average score per track of all of the players
* Determine which player is the best on any specific track
* A leaderboard to display the ordering of player seeding (power rankings)
* The ability to search for all the stats of a specifc track, including total points, player averages, and the individual track leaderboards
* The ability to search of all of the stats of a individual player, including total points, track averages, Track MVPs, and total power ranking points (Kart Score)
* The ability to correct mistakes in entering race points, by having a user setting to manually edit the points of a specific player
* Ability to view the database containing all of the data
* Ability to view a players seasonal stats and their all time stats. 
* Different seasonal leaderboards to show who is on top! 


**Important:** Make sure to run all cells before running the main cell, the main cell is the last one.


**Patch Notes**

v3.1 (6/20/22)
  - A new input (11) which allows the user to view all time stats
  - This feature combines all of the current seasons stats with those in the perminate all time file to get all kinds of total statics about a players performance over time. 
 
  Current stats include:
    1. Total Points
    2. Total Races
    3. Average Placement Points
    4. An estimation on GPs played
    5. GP Wins
    6. GP Win %
    7. Shock Dodges, Blue shells, and other misc stats

v3.0 (5/19/22)

- Seasons Update!
  - Seasons reset every couple of months to allow for play to be broken up during our time at college, currently there will be a summer and winter season
  - This allows for balances changes to be made each season and create a more fair ranking system overtime as rules change
  - An all time ranking will be determined by normializing each seasons points as a precent of their total, thus no matter the scoring system, the maximum possible points achieveavle is 100, before bonuses.
  -To incentivize placement there will be multipliers to the 1st, 2nd, and 3rd highest scorers. 1.25, 1.125, 1.075 respectivley. Thus the maximum possible amount of all-time points to get in a season is 125.
  -Each previous season will have their points reduced by 15% to ensure that the most recent season is the one with the most weight.
- Whats New?
  - A second database was created in google sheets to store all-time data
  - A user option to end the current season and start a new one
    - This function wipes the old season stats to zero and updates the all-time standings instead
  - A new algorythem for determining the best all time player
  - Balance Changes to the scoring system
    - Reducing Points for Track MVP from 2 to 0.25
    - Blue Shell Dodges 2 points
    - Blue Shell Hit .25 points
    - Shock Dodges Increased from 2 to 4 points


v2.3 (5/13/22)
- Blue Shells hit and Blue Shells Dodges implementation 
- Updated Kart Score and Seeding to include these metrics
- New Leaderboards for Blue Shells
- Added New Selection option for user entering Blue Shell Data
- Updated the Display Player Stats Function to include Blue Shells
- Re-orginized Kart Score using Constant variables to allow for quick balance adjustments in the future.
- Added Headings and sections so that the google collab Table of Contents was usuable
- Re-orginized code into these sections so that similar functions are in the same grouping, allows for ease of access


v2.2 (Jan 7 2022)
- Re-design of player stats output
- Re-orginization of code execution order for determining when the database is updated compared to finding track MVPs
-Points Per Race Leaderbaord
-A Feature to view all track MVPs at once
-Re-design of user input options
-Track Nickname functionality for user inputs (currently 77 working track nicknames)


v2.1 (Jan 4 2022)
- Optimization
- Changes to 'Kart Score' 
- MVP Leaderboards Per Track

v2.0 (Jan 3 2022)
 - Adds Shock Dodges 
 - Adds GP Wins
 - Adds Total Race Count
 - Leaderboards

v1.1 (Jan 2nd 2022)
 - Moved the Main Method into a private method, this reduced the need for scrolling

v1.0 (December 27 2021 - December 31st 2021)
 - Importing and Saving Race Data
 - Editing singular race scores incase of mistakes
 - Viewing Track Records + Best Player on the Track
 - Viewing Player Records


#Input Functions
  Where all of the input data functions are located


In [1]:
def inputRace(currentData,RaceCount,dfKartScore, Track, Racers, Scores,TrackIndex):  
  #checks to see if the track exists
  if Track in set(currentData["Tracks x Players"]):
    print('Valid', Track)
    #seperates the players into an array, make sure that all players also exist

    racersArray = Racers.split()
    # checks that all players are valid
    validCount = 0
    for racer in racersArray:
      if(racer in set(currentData.columns.values.tolist())):
          print('Valid', racer)
          validCount = validCount + 1
      else: 
          validCount = validCount - 1
          print('Invalid', racer)

  
    if validCount == len(racersArray):
      print("....Everything is valid, Updating the scores....")

      ##if all players are valid update the scores

      #gets scores into an array
      newScores = Scores.split()
      #changes to integers
      for i in range(len(newScores)):
        newScores[i] = int(newScores[i])
      #Adds new Score to the current score for each racer, uses the track index to find the track
      for i in range(len(racersArray)):
        
        #only gives the points if there is a singular MVP
        if(len(getTrackOwner(currentData,RaceCount, Track, TrackIndex).split()) == 1):
        #gives the track owner a tally for each person that played the track
          dfKartScore.at[0, getTrackOwner(currentData,RaceCount, Track, TrackIndex)] = int(dfKartScore.at[0, getTrackOwner(currentData,RaceCount, Track, TrackIndex)]) + 1


        #increments the score
        currentData.at[TrackIndex[Track],racersArray[i]] =  int(currentData.at[TrackIndex[Track],racersArray[i]]) + newScores[i]
        #increments the count
        RaceCount.at[TrackIndex[Track],racersArray[i]] =  int(RaceCount.at[TrackIndex[Track],racersArray[i]]) + 1
       
        
    else: 
      print('There was an error entering the racers, please try again')
  else:
    print('There was an error entering the track name, please try again:', Track, "was invalid")





In [2]:
def editAScore(dfScores, Track, Racer, Score,TrackIndex):
 #makes sure track is valid
  if Track in set(dfScores["Tracks x Players"]):
    print('Valid', Track)
    # makes sure racer is valid
    if Racer in set(dfScores.columns.values.tolist()):
      print('Valid', Racer)
      #changes a specific score
      dfScores.at[TrackIndex[Track],Racer] =  int(dfScores.at[TrackIndex[Track],Racer]) + int(Score)
      print(Racer, '\'s Score was changed by ', Score, ' on ', Track)
    else:  
      print('There was an error entering the racer, please try again')
  else:
     print('There was an error entering the track name, please try again:', Track, "was invalid")



In [3]:
#allows for GP winners to be counted
def enterWinner(dfWins, Player):

  if Player in set(dfWins.columns.values.tolist()):
    
    dfWins.at[0, Player] = int(dfWins.at[0,Player]) + 1
    print(Player, 'has', dfWins.at[0,Player], 'wins')


  else: 
    print(Player, "does not exist within the Database, Please try again")



In [4]:
#allowes for Shock dodges
def enterDodges(dfShock, Player, Count):
  if Player in set(dfShock.columns.values.tolist()):
    
    dfShock.at[0, Player] = int(dfShock.at[0,Player]) + int(Count)
    print(Player, 'has', dfShock.at[0,Player], 'shock dodges')


  else: 
    print(Player, "does not exist within the Database, Please try again")


In [5]:
def addBlueShells(dfBlue, Player, hit, dodge):
  if Player in set(dfBlue.columns.values.tolist()):
    
    dfBlue.at[0, Player] = int(dfBlue.at[0,Player]) + int(hit)
    print(Player, 'has been hit by', dfBlue.at[0,Player], 'Blue Shells')
    dfBlue.at[1, Player] = int(dfBlue.at[1,Player]) + int(dodge)
    print(Player, 'has', dfBlue.at[1,Player], 'Blue Shell Dodges')
    
  else: 
    print(Player, "does not exist within the Database, Please try again")


# Stat Getter Functions

In [6]:
def GetTrackData(dfScores,dfRaceCount,Track,TrackIndex):
 
  #eliminates the first row, to allow for players only
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
 
  #make sure that the track exists in the data
  if Track in set(dfScores["Tracks x Players"]):
    print("Stats For: ", Track)
    #now display the data for the track plus any interesting stats
    print("\n", "Total Scores")
    print(dfScores.loc[[TrackIndex[Track]]])

    races = 0
    for player in set(dfNoTracks.columns.values.tolist()):
      races = races + int(dfRaceCount.at[TrackIndex[Track], player])
    print('\nTimes Played:',races )

    #prints player averages. 
    print('\nPlayer Averages:')
    for racer in set(dfNoTracks.columns.values.tolist()):
      print(racer,": ", getPlayerAverage(dfScores,dfRaceCount,racer,Track,TrackIndex))
    print('\n')
    print(getTrackOwner(dfScores, dfRaceCount, Track, TrackIndex), "is the MVP for this track")
    
    print('\nMVP Leaderboard')
    #determine the score for each player within the data
    dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
    #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'MVP Points':[]})
    counter = 0
   
    #gets the ownership scores of each player for the track
    TrackTotalPoints = 0
    for player in set(dfNoTracks.columns.values.tolist()):
      TrackTotalPoints = TrackTotalPoints + int(dfScores.at[TrackIndex[Track], player])
    for player in set(dfNoTracks.columns.values.tolist()):
      ownershipScore = ((int(dfScores.at[TrackIndex[Track],player])/TrackTotalPoints)*100) *.28 + getPlayerAverage(dfScores,dfRaceCount,player,Track,TrackIndex) *.72


      #puts the score into the datafram
      dfLeaderboard.loc[counter] = [player, ownershipScore]
      counter = counter + 1
      playerScore = 0

    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['MVP Points', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)

  else:
    print('Track Selection Was Invalid')


In [7]:
def getTrackOwner(dfScores,dfRaceCount, Track, TrackIndex):
  #eliminates the first row, to allow for players only
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]


  #goes through all players and then find the max score

  currentMaxScore = 0
  currentPlayer = ""
  TrackTotalPoints = 0
  for player in set(dfNoTracks.columns.values.tolist()):
    TrackTotalPoints = TrackTotalPoints + int(dfScores.at[TrackIndex[Track], player])

  if TrackTotalPoints != 0:
    for player in set(dfNoTracks.columns.values.tolist()):
      #way to calculate track owner, total points + average
      playerScore = ((int(dfScores.at[TrackIndex[Track],player])/TrackTotalPoints)*100) *.28 + getPlayerAverage(dfScores,dfRaceCount,player,Track,TrackIndex) *.72
  
      if playerScore > currentMaxScore:
        currentPlayer = player
        currentMaxScore = playerScore
      elif playerScore == currentMaxScore:
        currentPlayer = currentPlayer + ", " + player
      else:
        continue

  return currentPlayer



In [8]:
def getAllTrackOwners(dfScores,dfRaceCount,TrackIndex):
  #the list that will get all of the track mvps
  dfList = pd.DataFrame({'Track': [], 'Current MVP': []})

  counter = 0
  for track in TrackIndex:
    mvp = getTrackOwner(dfScores,dfRaceCount, track, TrackIndex)
    dfList.loc[counter] = [track,mvp]
    counter = counter + 1


  print(dfList)

In [9]:
def getPlayerAverage(dfScores, dfRaceCount, Player, Track, TrackIndex):
  #eliminates the first row, to allow for players only
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
  
  if Player in set(dfNoTracks.columns.values.tolist()):
    #checks for divide by 0

    RaceCount = int(dfRaceCount.at[TrackIndex[Track],Player])
    Score =  int(dfScores.at[TrackIndex[Track],Player])
   
    if(Score == 0 or RaceCount == 0):
      averageScore = 0
    else: 
      averageScore = Score/RaceCount
  
    

  else:
    averageScore = "ERROR"
    print("There was an internal with Averaging Have Pat re-read code")
  

  return averageScore



# Player Stats and Seeding, Score Calculation

In [10]:
#Point Values, constants which can be adjusted for ease of balancing
POINTS_SCORED_POINTS = 0.35        #points for each GP point scored
GP_WINS_POINTS = 100               #points per gp win
OWNER_POINTS = 0.25                #points per player on owned track
DODGE_POINTS = 3.5                 #points per dodge
AVERAGE_POINTS = 3                 #points for track average
BLUE_POINTS = 0.65                 #points for getting hit with blue shell
BLUE_D_POINTS = 9                  #points for dodging a blue shell



def getPlayerStats(dfScores,dfRaceCount, dfWins,dfShock, dfKartScore, dfBlue, Player, TrackIndex):
  
  print("Displaying All Seasonal Stats for", Player)
  print("\n")
  print("---------------")
  #gets all of the tracks that the player owns
  Owns = ''
  for track in TrackIndex:
    if getTrackOwner(dfScores,dfRaceCount,track,TrackIndex) == Player:
      
      if Owns == '':
        Owns = track
      else:
        Owns = Owns + "\n" + track

  if(Owns == ''):
    print("Track MVP on: None")
  else:
    print("Tracks MVP on:\n", Owns)

  print("---------------")

  #prints total wins
  totalRaces = 0
  for Track in TrackIndex:
    totalRaces = totalRaces + int(dfRaceCount.at[TrackIndex[Track],Player])
  print("Total Races Played:", totalRaces)

  #Gp Wins
  print("Grand Prix Wins:", dfWins.at[0,Player])
  #dodges
  print("Shock Dodges:", dfShock.at[0,Player])
  #blue shell hits
  print("Times hit with a Blue:", dfBlue.at[0,Player])
  #blue shell dodges
  print("Blue Dodges:", dfBlue.at[1,Player])

  ####KART SCORE####
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
  KartScore = 0
  
  #gp wins points
  WinsScore = int(dfWins.at[0,Player])*GP_WINS_POINTS
    
  #owner score 
  OwnedScore = int(dfKartScore.at[0, Player]) * OWNER_POINTS
     
  #Dodges
  DodgeScore = int(dfShock.at[0,Player])*DODGE_POINTS
    
  #total points
  PointsScore = 0
  for track in TrackIndex:
   PointsScore = PointsScore + (int(dfScores.at[TrackIndex[track], Player])*POINTS_SCORED_POINTS)

  #adds in track averages
  avgScore = 0
  for track in TrackIndex:
    avgScore = avgScore + (int(getPlayerAverage(dfScores,dfRaceCount,Player,track,TrackIndex))*AVERAGE_POINTS)
  
  #points for blue shells
  blueScore = (int(dfBlue.at[0,Player]) * BLUE_POINTS) + (int(dfBlue.at[1,Player]) * BLUE_D_POINTS)



  KartScore = WinsScore + OwnedScore + DodgeScore + avgScore  + PointsScore + blueScore
  print('---------------')
  print("Kart Score:", KartScore)
  print('---------------')
  print("Kart Score Breakdown:")
  print('GP Win Points:', WinsScore)
  print('Points From Being Track MVP:', OwnedScore)
  print('Race Points:', PointsScore)
  print('Track Avg Points:', avgScore)
  print('Shock Dodge Points:', DodgeScore)
  print('Blue Shell Points:', blueScore)

  #prints the total Scores
  dfPlayerScores = dfScores[["Tracks x Players", Player]]
  print(Player, "\'s Total Scores")
  print("\n")
  print(dfPlayerScores)

  #prints Averages
  print("\n")
  print(Player, "\'s Track Averages")
  print("\n")
  for track in TrackIndex:
    print(track, "- ", getPlayerAverage(dfScores,dfRaceCount,Player,track,TrackIndex))

def getSeedings(sh,TrackIndex):
    
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    Wins = sh.worksheet('GP Wins').get_all_values()
    Shock = sh.worksheet('Shock Dodges').get_all_values()
    KartScore = sh.worksheet('Owned Score').get_all_values()
    Blue = sh.worksheet('Blue Shells').get_all_values()

    dfKartScore = pd.DataFrame(KartScore[1:], columns = KartScore[0])
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    dfShock = pd.DataFrame(Shock[1:], columns = Shock[0])
    dfBlue = pd.DataFrame(Blue[1:], columns = Blue[0])

    #determine the score for each player within the data
    dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
    #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'Kart Score':[]})
    counter = 0
   
    #sets up the scores for each player
    for player in set(dfNoTracks.columns.values.tolist()):
      playerScore = 0
      #gp wins 
      playerScore = int(dfWins.at[0,player])*GP_WINS_POINTS
    
      #enters the owned track score 
      playerScore = playerScore + int(dfKartScore.at[0, player])*OWNER_POINTS


      #Dodges
      playerScore = int(dfShock.at[0,player])*DODGE_POINTS + playerScore
    
      #total points
      points = 0
      for track in TrackIndex:
        points = points + int(dfScores.at[TrackIndex[track], player]) * POINTS_SCORED_POINTS
      playerScore = playerScore + points

      #adds in track averages
      avgpoints = 0
      for track in TrackIndex:
        avgpoints = avgpoints + (int(getPlayerAverage(dfScores,dfRaceCount,player,track,TrackIndex))*AVERAGE_POINTS)
      playerScore = playerScore + avgpoints

      #adds blue points
      playerScore = playerScore + (int(dfBlue.at[0,player]) * BLUE_POINTS) + (int(dfBlue.at[1,player]) * BLUE_D_POINTS)

      #puts the score into the datafram
      dfLeaderboard.loc[counter] = [player, playerScore]
      counter = counter + 1
      playerScore = 0

    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['Kart Score', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)

    return dfLeaderboard

In [11]:
#This function will gather all of the stats for each player, all time.
#this is opposed to just the seasonal stats

#This function will get the data from the two different sheets and combine them into 
#a place where a given player can see all of their stats
def getAllTimeStats(season,allTime,player,TrackIndex):


    print('Loading Data....')
    #seasonal stats
    kartData = season.worksheet('Total Scores').get_all_values()
    RaceCount = season.worksheet('Race Count').get_all_values()
    Wins = season.worksheet('GP Wins').get_all_values()
    Shock = season.worksheet('Shock Dodges').get_all_values()
    OwnedScore = season.worksheet('Owned Score').get_all_values()
    Blue = season.worksheet('Blue Shells').get_all_values()
    
    #alltime stats
    kartDataAllTime = allTime.worksheet('Total Scores').get_all_values()
    RaceCountAllTime = allTime.worksheet('Race Count').get_all_values()
    WinsAllTime = allTime.worksheet('GP Wins').get_all_values()
    ShockAllTime = allTime.worksheet('Shock Dodges').get_all_values()

    OwnedScoreAllTime = allTime.worksheet('Owned Score').get_all_values()

    BlueAllTime = allTime.worksheet('Blue Shells').get_all_values()
  

    print('Combining Old and New Data....')

    #the dataframes from each of the sheets (seasonal)
    dfSeasonOwnedScore = pd.DataFrame(OwnedScore[1:], columns = OwnedScore[0])
    dfSeasonScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfSeasonRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfSeasonWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    dfSeasonShock = pd.DataFrame(Shock[1:], columns = Shock[0])
    dfSeasonBlue = pd.DataFrame(Blue[1:], columns = Blue[0])

    #all time
    dfAllTimeOwnedScore = pd.DataFrame(OwnedScoreAllTime[1:], columns = OwnedScoreAllTime[0])
    dfAllTimeScores = pd.DataFrame(kartDataAllTime[1:], columns=kartDataAllTime[0])
    dfAllTimeRaceCount = pd.DataFrame(RaceCountAllTime[1:], columns = RaceCountAllTime[0])
    dfAllTimeWins = pd.DataFrame(WinsAllTime[1:], columns = WinsAllTime[0])
    dfAllTimeShock = pd.DataFrame(ShockAllTime[1:], columns = ShockAllTime[0])
    dfAllTimeBlue = pd.DataFrame(BlueAllTime[1:], columns = BlueAllTime[0])


    #removing titles
   # dfAllTimeOwnedScore =  dfAllTimeOwnedScore[dfAllTimeOwnedScore.columns.difference(["Tracks x Players"])]
    #dfAllTimeBlue = dfAllTimeBlue[dfAllTimeBlue.columns.difference(['Blue Shell'])]
    #dfAllTimeShock =  dfAllTimeShock[dfAllTimeShock.columns.difference(["Tracks x Players"])]
    #dfAllTimeShock =  dfAllTimeShock[dfAllTimeShock.columns.difference(["Tracks x Players"])]

    print('Doing Calculations...')

    if player == 'all':
      dfAllTimeWins = dfAllTimeWins[dfAllTimeWins.columns.difference(["Tracks x Players"])]
      players = dfAllTimeWins.columns
    else:
      players = []
      players.append(player)


    for player in players:
    #combine the data frames into one df for use later in displaying the stats
      for track in TrackIndex:
        dfAllTimeScores.loc[TrackIndex[track], player] = int(dfAllTimeScores.at[TrackIndex[track], player]) + int(dfSeasonScores.at[TrackIndex[track], player])
        dfAllTimeRaceCount.loc[TrackIndex[track], player] = int(dfAllTimeRaceCount.at[TrackIndex[track], player]) + int(dfSeasonRaceCount.at[TrackIndex[track], player])
      
      #others not related to tracks
      dfAllTimeOwnedScore.loc[0,player] = float(dfAllTimeOwnedScore.loc[0,player]) + float(dfSeasonOwnedScore.loc[0,player])
      dfAllTimeWins.loc[0,player] = int(dfAllTimeWins.loc[0,player]) + int(dfSeasonWins.loc[0,player])
      dfAllTimeShock.loc[0,player] = int(dfAllTimeShock.loc[0,player]) + int(dfSeasonShock.loc[0,player])
    
      #blueshell is two columns
      dfAllTimeBlue.loc[0,player] = int(dfAllTimeBlue.loc[0,player]) + int(dfSeasonBlue.loc[0,player])
      dfAllTimeBlue.loc[1,player] = int(dfAllTimeBlue.loc[1,player]) + int(dfSeasonBlue.loc[1,player])
    

      #players total points scores, and races
      totalPoints = 0
      totalRaces = 0
      for track in TrackIndex:
        totalPoints = totalPoints + int(dfAllTimeScores.loc[TrackIndex[track],player])
        totalRaces = totalRaces + int(dfAllTimeRaceCount.loc[TrackIndex[track],player])

        #fixes divide by 0 error
        if totalRaces == 0:
          average = 0
          FirstPlaceRate = 0
        else:
          average = totalPoints/totalRaces
          FirstPlaceRate = (int(dfAllTimeWins.at[0,player]) / (totalRaces/8))*100

        ##Display formatting and print statements
      print('\n\nDisplaying the All-Time Stats for', player)
      print('\n---Race Stats---')
      print('Total Race Points: ', totalPoints)
      print('Total Race Count: ', totalRaces)
      print('Average Placement Points: ', average)

      print('\n---GP Stats---')
      print('Total GP Wins:', dfAllTimeWins.at[0,player])
      print('Total GPs Played:', totalRaces/8)
      print('First Place Rate: ', FirstPlaceRate , '%'  )

      print('\n---Misc Stats---')
      print('Shock Dodges:', dfAllTimeShock.at[0,player])
      print('Times Hit By A Blue Shell:', dfAllTimeBlue.at[0,player])
      print('Blue Dodges: ', dfAllTimeBlue.at[1,player])
      print('Races Played on Owned Track', float(dfAllTimeOwnedScore.at[0,player])/4)
    



#Leaderboard Generators

In [12]:
def getShockDodges(dfShock):
  #dataframe without the label
    dfNoLabel = dfShock[dfShock.columns.difference(["Tracks x Players"])]
  #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'Shock Dodges':[]})
    counter = 0
    #for each player in dodges
    for player in set(dfNoLabel.columns.values.tolist()):
      #adds the dodges to the leaderboard
      dfLeaderboard.loc[counter] = [player, int(dfNoLabel.at[0, player])]
      counter = counter + 1
      racecount = 0
   
    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['Shock Dodges', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)



In [13]:
def getRaceCountLeaderbaords(dfRaceCount,TrackIndex):
  
    #determine the score for each player within the data
    dfNoTracks = dfRaceCount[dfRaceCount.columns.difference(["Tracks x Players"])]
    #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'Races Played':[]})
    counter = 0
   
    racecount = 0
    #sets up the scores for each player
    for player in set(dfNoTracks.columns.values.tolist()):
      
      for track in TrackIndex:
        racecount = racecount + int(dfRaceCount.at[TrackIndex[track], player])


      #adds the racecount to the leaderboard
      dfLeaderboard.loc[counter] = [player, racecount]
      counter = counter + 1
      racecount = 0
   
    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['Races Played', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)


In [14]:
def getGPWinsLeaderboard(dfWins):
    #dataframe without the label
    dfNoLabel = dfWins[dfWins.columns.difference(["Tracks x Players"])]

    #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'Grand Prix Wins':[]})
    counter = 0
    #for each player in wins
    for player in set(dfNoLabel.columns.values.tolist()):
      #adds the wins to the leaderboard
      dfLeaderboard.loc[counter] = [player, int(dfNoLabel.at[0, player])]
      counter = counter + 1
      racecount = 0
   
    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['Grand Prix Wins', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)


In [15]:
def getPointsPerRace(dfScores,dfRaceCount, TrackIndex):
  #creates a leaderboard object
  dfLeaderboard = pd.DataFrame({'Player': [], 'Points Per Race': []})
  counter = 0


   #dataframe without the track names
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]

  #iterates through each player
  for player in set(dfNoTracks.columns.values.tolist()):

    #gets the total points of each player
    totalPoints = 0
    for track in TrackIndex:
      totalPoints = totalPoints + int(dfScores.at[TrackIndex[track],player])
    #gets total racecount
    totalRaces = 0
    for track in TrackIndex:
      totalRaces = totalRaces + int(dfRaceCount.at[TrackIndex[track],player])

    #gets the PPR and then sets it to the approperate place
    
    if (totalRaces != 0):
      PPR = float(totalPoints) / float(totalRaces)
      dfLeaderboard.loc[counter] = [player, PPR]
      counter = counter + 1
    else:
      dfLeaderboard.loc[counter] = [player,0]
      counter = counter + 1
     
    

   #sorts and prints the leaderboard
  dfLeaderboard = dfLeaderboard.sort_values(['Points Per Race', 'Player'], ascending=[0,1]) 
  print(dfLeaderboard)
    




In [16]:
def getBlueLeaderboard(dfBlue):
     #dataframe without the label
    dfNoLabel = dfBlue[dfBlue.columns.difference(["Blue Shell"])]
    
    #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'Blue Shells':[]})

    counter =0
    #for each player 
    for player in set(dfNoLabel.columns.values.tolist()):
      #adds the blues to the leaderboard
      dfLeaderboard.loc[counter] = [player, int(dfNoLabel.at[0, player]) + int(dfNoLabel.at[1,player])]
      counter = counter + 1
      racecount = 0
   
    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['Blue Shells', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)

# Season Reset Code

In [17]:
from pandas.core.dtypes.cast import sanitize_to_nanoseconds
#this code takes in the name of the files, and then the sheet name, and then transfers all the data
def transfer_data(old,new,name):
  oldKart = old.worksheet(name).get_all_values()
  newKart= new.worksheet(name).get_all_values()
  dfOldKart = pd.DataFrame(oldKart[1:], columns = oldKart[0])
  dfNewKart = pd.DataFrame(newKart[1:], columns = newKart[0])

  #deleats first column
  colList = dfOldKart.columns.values.tolist();
  del colList[0]
 
  #iterates through the sheets
  for col in colList:
    for row in dfOldKart.index:
        dfNewKart.at[row,col] = int(dfNewKart.at[row,col]) + int(dfOldKart.at[row,col])
        #clears the old stats
        dfOldKart.at[row,col] = 0
 
  return dfNewKart,dfOldKart;


#This function will take all of the seasonal stats that have been aquired, 
#increment them into the all time sheet, and the reset the season for the next use
def end_season(season, all_time,TrackIndex):
  #season is the sheet for the current season
  #all time is the all time stats'

  #first get the seeding points and then update the kart sheet
  dfFinalRanks = getSeedings(season,TrackIndex)

  #find the score of each player using the decided algorythem, and then 
  #increment this score with the all time score
  
  totalScores = 0
  #get the total points,
  for index in dfFinalRanks.index:
    totalScores += dfFinalRanks.at[index,'Kart Score']

  
  #make a new leaderboard, and then add all the players to it with their new scores:
  dfSeasonScores = pd.DataFrame({'Player': [], 'Season Score':[]})
  for index in dfFinalRanks.index:
    #make score percent of score of the total:
    score = (int(dfFinalRanks.at[index,'Kart Score']) / totalScores) * 100
    #multiplier for placements
    if(index == 0):
      score *= 1.25
    elif(index == 1):
      score *= 1.125
    elif(index == 2):
      score *= 1.05

    dfSeasonScores.at[index] = [dfFinalRanks.at[index,'Player'],score]
    
  print("Below are the Final Scores Normalized for the Season")
  print(dfSeasonScores)


  #adds the new df to the all time scoring sheet
  all_timeScores= all_time.worksheet('All-Time Seeding').get_all_values()
  dfAll_timeScores = pd.DataFrame(all_timeScores[1:], columns = all_timeScores[0])  


  print('Updating All Time Seeding....')
  print('Diminishing Old Seasons by 15%...')
  print('Incrementing The New Seasons Scores...')
  
  for index in dfSeasonScores.index:
    player = dfSeasonScores.at[index,'Player']

  
    dfAll_timeScores.at[0,player] = (float(dfAll_timeScores.at[0,player])*.85) + float(dfSeasonScores.at[index,'Season Score'])

  #increments the all time stats with the seasonal stats, and then clears all of the seasonal stats

  #calls the transfer function on points score
  newPoints,oldPoints = transfer_data(season,all_time,'Total Scores')
  
  #calls it for the race count
  newRaceCount,oldRaceCount = transfer_data(season,all_time,'Race Count')

  #GP Wins
  newWins,oldWins = transfer_data(season,all_time,'GP Wins')

  #Shock Dodges
  newDodge,oldDodge = transfer_data(season,all_time,'Shock Dodges')

  #races owned
  newOwned,oldOwned = transfer_data(season,all_time,'Owned Score')

  #blue Shells
  newShells,oldShells = transfer_data(season,all_time,'Blue Shells')


  return newPoints,newRaceCount,newWins,newDodge,newOwned,newShells,dfAll_timeScores,oldPoints,oldRaceCount,oldWins,oldDodge,oldOwned,oldShells

 


# I/O Logic

In [18]:
#This is the main bulk of the code, it culminates all of the previous functions 
#into the user inputted choices, this is the method that gets user inputs and does all 
#of the database scraping and saving for this program to work.
def RunKartniteStats():
### Patrick Marinich December 2021

  #constants
  VERSION_NUMBER = 'v3.1'
  CONTRIBUTORS = 'Patrick Marinich'


  #imports
  import pandas as pd
  from google.colab import auth
  auth.authenticate_user()
  import gspread
  from gspread_dataframe import get_as_dataframe, set_with_dataframe

  from google.auth import default
  creds, _ = default()

  gc = gspread.authorize(creds)

  # Puts the google sheet into a python object, we will be calling the sh object after a selection is made
  sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1G0z4XeBjG7Q_Zk_uPtV4v8BPkVT8GD9pmsuk7VIwFKc/edit#gid=1608384215')

  #Track Dictionary, all track names and nicknames go here
  #track indexes are starting at 0 in the mushroom cup ending at 31 at N64 Bowsers castle, +32 for Lava Lake
  TrackIndex = {"Luigi Circuit":0,
  "Moo Moo Meadows":1,
  "Mushroom Gorge":2,
  "Toad's Factory":3,
  "Mario Circuit":4,
  "Coconut Mall":5,
  "DK Summit":6,
  "Wario's Gold Mine":7,
  "Daisy Circuit":8,
  "Koopa Cape":9,
  "Maple Treeway":10,
  "Grumble Volcano":11,
  "Dry Dry Ruins":12,
  "Moonview Highway":13,
  "Bowser's Castle":14,
  "Rainbow Road":15,
  "GCN Peach Beach":16,
  "DS Yoshi Falls":17,
  "SNES Ghost Valley 2":18,
  "N64 Mario Raceway":19,
  "N64 Sherbet Land":20,
  "GBA Shy Guy Beach":21,
  "DS Delfino Square":22,
  "GCN Waluigi Stadium":23,
  "DS Desert Hills":24,
  "GBA Bowser's Castle 3":25,
  "N64 DK's Jungle Parkway":26,
  "GCN Mario Circuit":27,
  "SNES Mario Circuit 3":28,
  "DS Peach Gardens":29,
  "GCN DK Mountain":30,
  "N64 Bowser's Castle":31,
  "Lava Lake":32 }

#for nicknames 
  NickNameIndex= {"Luigi":"Luigi Circuit",
   "moo moo": "Moo Moo Meadows",
   "Moo Moo": "Moo Moo Meadows",
   "Gorge":"Mushroom Gorge",
   "gorge":"Mushroom Gorge",
   "toads":"Toad's Factory",
   "Toads":"Toad's Factory",
   "toads factory": "Toad's Factory",
   "mario circuit": "Mario Circuit",
   "coconut mall":"Coconut Mall",
   "coconut":"Coconut Mall",
   "summit":"DK Summit",
   "Summit":"DK Summit",
   "gold mine":"Wario's Gold Mine",
   "Gold Mine":"Wario's Gold Mine",
   "koopa": "Koopa Cape",
   "Koopa": "Koopa Cape",
   "maple":"Maple Treeway",
   "Maple":"Maple Treeway",
   "grumble":"Grumble Volcano",
   "Grumble":"Grumble Volcano",
   "Dry Dry":"Dry Dry Ruins",
   "dry dry":"Dry Dry Ruins",
   "Moonview":"Moonview Highway",
   "moonview":"Moonview Highway",
   "BC Wii":"Bowser's Castle",
   "bc wii":"Bowser's Castle",
   "BC wii":"Bowser's Castle",
   "rainbow road":"Rainbow Road",
   "rainbow":"Rainbow Road",
   "Rainbow":"Rainbow Road",
   "Peach Beach":"GCN Peach Beach",
   "peach beach":"GCN Peach Beach",
   "yoshi falls":"DS Yoshi Falls",
   "Yoshi Falls":"DS Yoshi Falls",
   "Ghost Valley":"SNES Ghost Valley 2",
   "ghost valley":"SNES Ghost Valley 2",
   "mario raceway":"N64 Mario Raceway",
   "raceway": "N64 Mario Raceway",
   "Raceway": "N64 Mario Raceway",
   "Sherbet Land":"N64 Sherbet Land",
   "sherbet land":"N64 Sherbet Land",
   "Shy Guy Beach":"GBA Shy Guy Beach",
   "shy guy beach":"GBA Shy Guy Beach",
   "Shy Guy":"GBA Shy Guy Beach",
   "shy guy":"GBA Shy Guy Beach",
   "Delfino Square":"DS Delfino Square",
   "delfino square":"DS Delfino Square",
   "Delfino":"DS Delfino Square",
   "delfino":"DS Delfino Square",
   "Waluigi Stadium":"GCN Waluigi Stadium",
   "waluigi stadium":"GCN Waluigi Stadium",
   "Waluigi":"GCN Waluigi Stadium", 
   "waluigi":"GCN Waluigi Stadium",
   "Desert Hills":"DS Desert Hills",
   "desert hills":"DS Desert Hills",
   "GBA 3":"GBA Bowser's Castle 3",
   "gba 3":"GBA Bowser's Castle 3",
   "BC3":"GBA Bowser's Castle 3",
   "bc3":"GBA Bowser's Castle 3",
   "Parkway":"N64 DK's Jungle Parkway",
   "parkway":"N64 DK's Jungle Parkway",
   "GCN mario circuit":"GCN Mario Circuit",
   "GCN mario":"GCN Mario Circuit",
   "gcn mario":"GCN Mario Circuit",
   "gcn Mario":"GCN Mario Circuit",
   "SNES 3":"SNES Mario Circuit 3",
   "snes 3":"SNES Mario Circuit 3",
   "Peach Gardens":"DS Peach Gardens",
   "peach gardens":"DS Peach Gardens",
   "DK Mountain": "GCN DK Mountain",
   "dk mountain": "GCN DK Mountain",
   "mountain": "GCN DK Mountain",
   "Mountain": "GCN DK Mountain",
   "BC64":"N64 Bowser's Castle",
   "bc64":"N64 Bowser's Castle",
   "bc 64":"N64 Bowser's Castle",
   "N64BC":"N64 Bowser's Castle",
   "n64bc":"N64 Bowser's Castle"}

  #Below is the logic for asking a user what they would like to do
  ########


  #displaying inputs for the user
  print('Welcome to Kartnite Stats ' ,VERSION_NUMBER , "\nDeveloped by:", CONTRIBUTORS, '\n')
  print('Below are the options, type the number of the option you would like')
  print('1. Input a Race \n2. Edit A Singular Score \n3. Enter A Grand Prix Winner \n4. Enter Shock Dodges \n5. Enter Blue Shell Stats \n6. View Leaderboards \n7. View All Databases \n8. View Track Records \n9. View Player Stats  \n10. View All Current Track MVPs \n11. View Player All-Time Stats')
 
  selection = input('What Would You Like To Do: ')
  
  #selection logic
  
  #depending on the user selection it will ask for different inputs, and load the nessassary databases needed for that operation

  #input a race
  if selection == '1':
    print('Warining: track and racer names must be formatted correctally \nTrack names must match their full name or a registered nickname \nRacer names must have their first letter capital')
    Track = input("Name of Track: ")
    if Track in NickNameIndex:
      Track = NickNameIndex[Track]
    Racers = input("Names of Racers: ")
    Scores = input("Scores:")
    #loading statment
    print("\nYour selection of", selection, "is loading...." )
    print('\n\n')
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    KartScore = sh.worksheet('Owned Score').get_all_values()
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfKartScore = pd.DataFrame(KartScore[1:], columns = KartScore[0])
    inputRace(dfScores,dfRaceCount, dfKartScore, Track,Racers,Scores,TrackIndex)
 
 #edit a score
  elif selection == '2':
    print('Edit A Score is used to fix an error that may have occured while entering a race \nUse a positive number to increase a score, and a negative to decrease \nUse the net change you want to make as the value')
    Track = input("Name of Track: ")
    if Track in NickNameIndex:
      Track = NickNameIndex[Track]
    Racer = input("Names of Racer: ")
    Score = input("Score: ")
    print("\nYour selection of", selection, "is loading...." )
    print('\n\n')
    kartData = sh.worksheet('Total Scores').get_all_values()
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    editAScore(dfScores, Track, Racer, Score,TrackIndex)

  #enter a winner
  elif selection == '3':
    Player = input("Winning Player: ")
    print("\nYour selection of", selection, "is loading...." )
    print('\n')
    Wins = sh.worksheet('GP Wins').get_all_values()
    dfWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    enterWinner(dfWins, Player)
   
  #enter a dodge
  elif selection == '4':
    Player = input("Player Name: ")
    Count = input("How Many Dodges: ")
    print("\nYour selection of", selection, "is loading...." )
    print('\n')
    Shock = sh.worksheet('Shock Dodges').get_all_values()
    dfShock = pd.DataFrame(Shock[1:], columns = Shock[0])
    enterDodges(dfShock,Player,Count)


  elif selection == '5':
    Player = input("Player Name: ")
    hit = input("How Many times where they hit by a blue shell? ")
    dodge = input("How many did they dodge? ")
    print("\nYour selection of", selection, "is loading...." )
    print('\n')
    
    BlueShells = sh.worksheet("Blue Shells").get_all_values()
    dfBlueShell = pd.DataFrame(BlueShells[1:], columns = BlueShells[0])

    addBlueShells(dfBlueShell, Player, hit, dodge)

   #leaderboards
  elif selection == '6':
    print("\nYour selection of", selection, "is loading...." )
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    Wins = sh.worksheet('GP Wins').get_all_values()
    Shock = sh.worksheet('Shock Dodges').get_all_values()
    KartScore = sh.worksheet('Owned Score').get_all_values()
    Blue = sh.worksheet('Blue Shells').get_all_values()

    dfKartScore = pd.DataFrame(KartScore[1:], columns = KartScore[0])
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    dfShock = pd.DataFrame(Shock[1:], columns = Shock[0])
    dfBlue = pd.DataFrame(Blue[1:], columns = Blue[0])
    print('\n\n')
   
    print('Seeding Leaderboard')
    getSeedings(sh,TrackIndex)
    print('\nPoints Per Race Leaderboard')
    getPointsPerRace(dfScores,dfRaceCount,TrackIndex)
    print('\nTotal Races Played')
    getRaceCountLeaderbaords(dfRaceCount, TrackIndex)
    print('\nGP Wins')
    getGPWinsLeaderboard(dfWins)
    print('\nShock Dodges')
    getShockDodges(dfShock)
    print('\n Blue Shells')
    getBlueLeaderboard(dfBlue)
 
  #print all data
  elif selection == '7':
    print("\nYour selection of", selection, "is loading...." )
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    Wins = sh.worksheet('GP Wins').get_all_values()
    Shock = sh.worksheet('Shock Dodges').get_all_values()
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    dfShock = pd.DataFrame(Shock[1:], columns = Shock[0])

    print('\nDatabase of total scores')
    print(dfScores)
    print('\nDatabase of Race Count')
    print(dfRaceCount)
    print('\nDatabase GP Wins')
    print(dfWins)
    print('\nDatabase of Shock Dodges')
    print(dfShock)
 
  #track data
  elif selection == '8':
    Track = input("Name of Track: ")
    if Track in NickNameIndex:
      Track = NickNameIndex[Track]
    print("\nYour selection of", selection, "is loading...." )
    print("\n\n")
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    GetTrackData(dfScores,dfRaceCount,Track,TrackIndex)
  
  #racer data
  elif selection == '9':
    Racer = input("Name of Racer: ")
    print("\nYour selection of", selection, "is loading...." )
    print("\n\n")

    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    Wins = sh.worksheet('GP Wins').get_all_values()
    Shock = sh.worksheet('Shock Dodges').get_all_values()
    KartScore = sh.worksheet('Owned Score').get_all_values()
    Blue = sh.worksheet('Blue Shells').get_all_values()
    dfKartScore = pd.DataFrame(KartScore[1:], columns = KartScore[0])
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    dfShock = pd.DataFrame(Shock[1:], columns = Shock[0])
    dfBlue = pd.DataFrame(Blue[1:], columns = Blue[0])
    
    getPlayerStats(dfScores, dfRaceCount, dfWins, dfShock, dfKartScore, dfBlue, Racer, TrackIndex)
 
  #All Track MVPs
  elif selection == '10':
    print("\nYour selection of", selection, "is loading...." )
    print('\n')
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    print("Displaying all track MVPs...\n")

    getAllTrackOwners(dfScores,dfRaceCount,TrackIndex)

  elif selection == 'reset':
    answer = input("Are you sure you want to end the current season? ")
    
    if answer == 'yes' or answer == 'YES':
      #open up all time sheet
      change = 0
      all_time = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nolol4P63e7WY-wRPnogcePkWuPPE3JF6QBRgXgF8gE/edit#gid=524408434')
      print("Ending Season....")
      #call end of season function
      newPoints,newRaceCount,newWins,newDodge,newOwned,newShells,newSeeds,oldPoints,oldRaceCount,oldWins,oldDodge,oldOwned,oldShells = end_season(sh,all_time,TrackIndex)
      #prompt to end the season
      save = input("are you sure you want to save? ")
      if save == 'YES' or save == 'yes':
        #if yes set change to = 1
        change = 1
        print('Finializing the data....')
      else:
        print('Canceling Change of Season...')
        print('End of Program')
    else:
      print('Stopping program')
    
  #view the all time stats for a player
  elif selection == '11':
    #gets inputs for player(s)
    Racer = input("Name of Racer (type 'all' to view everybody's): ")
    #output formatting
    print("\nYour selection of", selection, "is loading...." )
    print("\n\n")
    #gets the all time sheet open
    all_time = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nolol4P63e7WY-wRPnogcePkWuPPE3JF6QBRgXgF8gE/edit#gid=524408434')
    
    #calls the function for all time stats
    getAllTimeStats(sh,all_time,Racer,TrackIndex)
  





   
 #invalid selection
  else: 
    print('invalid selection restart the program')



  ##COMMENT THIS OUT WHEN TESTING

  #upadates the google sheet, a psyudo save of the data
  TotalScores = sh.worksheet('Total Scores')
  RaceCount = sh.worksheet('Race Count')
  WinCount = sh.worksheet('GP Wins')
  Shock = sh.worksheet('Shock Dodges')
  KartScore = sh.worksheet('Owned Score')
  BlueShells = sh.worksheet('Blue Shells')
  
  #saves to the google sheet if nessassary, depending on the input of the user
  #saves if enter a race
  if selection =='1':
    set_with_dataframe(TotalScores, dfScores)
    set_with_dataframe(RaceCount,dfRaceCount)
    set_with_dataframe(KartScore,dfKartScore)
    print("Success - The Database Has Been Updated")
  #saves if edit a score
  elif selection =='2':
    set_with_dataframe(TotalScores, dfScores)
    print("Success - The Database Has Been Updated")
 #saves when winner added
  elif selection =='3':
    set_with_dataframe(WinCount,dfWins)
    print("Success - The Database Has Been Updated")
  #saves when shock dodges are counted
  elif selection =='4':
    set_with_dataframe(Shock,dfShock)
    print("Success - The Database Has Been Updated")
  elif selection  == '5':
    #saves the data for blue shells
    set_with_dataframe(BlueShells,dfBlueShell)
    print("Success - The Database Has Been Updated")

  elif selection == 'reset':
    if change == 1:
      #opens up the all time list
      print("Saving All-Time Stats")
      aTotalScores = all_time.worksheet('Total Scores')
      aRaceCount = all_time.worksheet('Race Count')
      aWinCount = all_time.worksheet('GP Wins')
      aShock = all_time.worksheet('Shock Dodges')
      aKartScore = all_time.worksheet('Owned Score')
      aBlueShells = all_time.worksheet('Blue Shells')
      aSeeds = all_time.worksheet("All-Time Seeding")
      #saves the data for the all time list
      set_with_dataframe(aTotalScores,newPoints)
      set_with_dataframe(aRaceCount,newRaceCount)
      set_with_dataframe(aWinCount,newWins)
      set_with_dataframe(aShock,newDodge)
      set_with_dataframe(aKartScore,newOwned)
      set_with_dataframe(aBlueShells,newShells)
      set_with_dataframe(aSeeds,newSeeds)



      #sets all od the old ones to 0
      print("Resetting Season Stats...")
      set_with_dataframe(TotalScores, oldPoints)
      set_with_dataframe(RaceCount,oldRaceCount)
      set_with_dataframe(KartScore,oldOwned)
      set_with_dataframe(WinCount,oldWins)
      set_with_dataframe(Shock,oldDodge)
      set_with_dataframe(BlueShells,oldShells)

      print("The Seasonal Stats have been successfully merged into the All-Time Stats")
      print("Time for a new season to begin!")



# Main

In [19]:
#This is now the cell you need to run the program
#The old main cell was redefined from the main cell to the singular method below
#This is to reduce the amount of scrolling needed when inputing or viewing the data

######IMPORTS######
import pandas as pd
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import get_as_dataframe, set_with_dataframe
###################

#methodCall
RunKartniteStats()

Welcome to Kartnite Stats  v3.1 
Developed by: Patrick Marinich 

Below are the options, type the number of the option you would like
1. Input a Race 
2. Edit A Singular Score 
3. Enter A Grand Prix Winner 
4. Enter Shock Dodges 
5. Enter Blue Shell Stats 
6. View Leaderboards 
7. View All Databases 
8. View Track Records 
9. View Player Stats  
10. View All Current Track MVPs 
11. View Player All-Time Stats
What Would You Like To Do: 11
Name of Racer (type 'all' to view everybody's): all

Your selection of 11 is loading....



Loading Data....
Combining Old and New Data....
Doing Calculations...


Displaying the All-Time Stats for Chris

---Race Stats---
Total Race Points:  3786
Total Race Count:  343
Average Placement Points:  11.037900874635568

---GP Stats---
Total GP Wins: 5
Total GPs Played: 42.875
First Place Rate:  11.661807580174926 %

---Misc Stats---
Shock Dodges: 5
Times Hit By A Blue Shell: 22
Blue Dodges:  0
Races Played on Owned Track 42.1875


Displaying the All-Time St